In [33]:
import cv2
import numpy as np
from PIL import Image
import io
import numpy as np
from pillow_heif import register_heif_opener
from PIL import Image

p = r"../data/images/photo_kodak/kodim04.png"
img = cv2.imread(p, cv2.IMREAD_COLOR)

In [ ]:
def jpeg_compress(img, q):
    res, data = cv2.imencode(".jpg", img, [cv2.IMWRITE_JPEG_QUALITY, q])
    assert res
    decoded_img = cv2.imdecode(data, cv2.IMREAD_COLOR)
    return len(data) * 8, decoded_img


def jpeg2000_compress(image_array, quality=50):
    image = Image.fromarray(image_array)  # Convert NumPy array to PIL image
    
    buffer = io.BytesIO()
    
    # Save with JPEG2000 format and specified quality layer
    image.save(buffer, format="JPEG2000", quality_layers=[quality])

    jpeg2000_bytes = buffer.getvalue()
    size_in_bits = len(jpeg2000_bytes) * 8  # Convert bytes to bits
    
    # Decode the JPEG2000 image back to see compression artifacts
    buffer.seek(0)  # Reset buffer position
    decoded_image = Image.open(buffer)
    decoded_array = np.array(decoded_image)  # Convert back to NumPy array

    return size_in_bits, decoded_array


def heif_compress(img, q):
    # Ensure Pillow supports HEIC
    register_heif_opener()
    
    # Convert OpenCV image (numpy array) to PIL Image
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Save the image as HEIC in memory
    img_bytes = io.BytesIO()
    img_pil.save(img_bytes, format="HEIF", quality=q)
    
    # Get the size of the encoded image in bits
    bit_size = len(img_bytes.getvalue()) * 8
    
    # Decode the HEIC image back to OpenCV format
    img_bytes.seek(0)
    decoded_pil = Image.open(img_bytes)
    decoded_img = cv2.cvtColor(np.array(decoded_pil), cv2.COLOR_RGB2BGR)
    
    return bit_size, decoded_img


In [35]:
size_jpeg, dec_jpeg = jpeg_compress(img, 92)
size_jpeg

905192

In [36]:
def err(img1, img2):
    return np.abs(img1.astype(np.float32) - img2.astype(np.float32)).sum()

In [45]:
size_heic, dec_heic = heic_compress(img, 100)
size_heic

2347216

In [38]:
err(img, dec_jpeg), err(img, dec_heic)

(2404005.0, 1856161.0)

In [39]:
# size_j2k, dec_j2k = jpeg2000_compress(img, quality=11)
# size_j2k

In [40]:
# err(img, dec_jpeg), err(img, dec_j2k)